# Dask

Dask baut auf bewährten Modulen auf und erweitert diese um Möglichkeiten zur massiven Parallelisierung. So können mehrere NumPy Arrays oder Pandas Dataframes in entsprechenden Dask-Objekten zusammengefasst und für parallele Operationen bereitgestellt werden. Die Dask Objekte stellen große Teile der bakannten API (identisch zu NumPy Arrays oder Pandas Dataframes) bereit.

![image](images/Dask_Scale.svg)

## Dask Dashboard

Eine Übersicht über die von Dask gestarteten Parallelen Vorgänge und deren Auslastung kann über das Dask Dashboard eingesehen werden. Das Client Objekt aus dem dask.distributed Modul ermöglicht das Starten eines Dask Dashboards. Wird das initialisierte Client Objekt ausgegeben, so enthält die Ausgabe eine URL unter der das gestartete Dashboard abgerufen werden kann.

Wenn Jupter in Version 3.0 installiert ist oder zusätzlich Node.js (Version >= 12.0.0) und npm installiert sind, kann alternativ zur manuellen Nutzung auch das dask-labextensions Plugin in Jupyter installiert werden. Dies sorgt für eine Integration des Dask Dashboards in die Jupyter Oberfläche. Am linken Rand ist dann eine neue Schaltfläche "Dask" vorhanden. Über dies kann das Dask Dashboard erreicht werden, ohne dass hierfür eine separate URL aufgerufen werden muss.

In [9]:
from dask.distributed import Client
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='2GB')
client

/home/julian/jupyterlab/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36895 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://94.16.122.16:36895/status,
Dashboard: http://94.16.122.16:36895/status,Workers: 1
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: False
Comm: inproc://94.16.122.16/286813/17,Workers: 1
Dashboard: http://94.16.122.16:36895/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: inproc://94.16.122.16/286813/20,Total threads: 4
Dashboard: http://94.16.122.16:40727/status,Memory: 1.86 GiB
Nanny: None,


In [10]:
client.close()

Aktuell funktioniert das dask-labextensions Plugin am bwUniCluster noch nicht. Alternativ kann wie oben beschrieben direkt die URL des Dashboards genutzt werden. Hierfür muss über ssh der Port aus der URL aus dem Cluster nach außen weitergeleitet werden. Dies kann mit folgendem Befehl lokal am genutzten Rechner in einer Konsole durchgeführt werden. Der Port und die IP des Jupyter-Compute-Node können dabei der Dashboard-URL entnommen werden.

```bash
ssh -N -L <Port>:<Jupyter-Compute-Node>:<Port> <Hochschulkürzel>_<User-ID>@bwunicluster.scc.kit.edu
```

Nach Ausführen des ssh-Port-Forwardings kann am lokalen Rechner das Dask-Dashboard unter

```bash
http://localhost:<Port>/status
```

aufgerufen werden.

## Dask Array

Dask Array koordiniert mehrere NumPy Arrays und verteilt diese auf die zur Verfügung stehenden Ressourcen. So können Operationen verteilt auf mehrere Threads, Prozesse oder gar Nodes ausgeführt werden. Welche Operationen dabei möglich sind (welche Teile der NumPy Array API auch von Dask Array angeboten werden) kann der Dokumentation entnommen werden: https://docs.dask.org/en/latest/array-api.html.

Weitere Beispiele zu Dask Array: https://mybinder.org/v2/gh/dask/dask-examples/main?urlpath=lab/tree/array.ipynb

## Dask und SLURM

Um Dask in Kombination mit SLURM (dem Job-Scheduler des bwUniClusters) nutzen zu können, wird die Klasse SLURMCluster aus dem Modul dask_jobqueue benötigt. Damit dieses Modul zur Verfügung steht muss im jeweiligen Environment sowohl dask als auch dask_jobqueue installiert sein:

```bash
python3 -m pip install dask_jobqueue dask
```

Ist der IPython-Kernel aus einem entsprechend erweiterten Environment im Jupyter registriert, so kann dieser beim Start eines neuen Notebooks ausgewählt werden. Anschließend kann die SLURMCluster Klasse im Notebook importiert und zum Erstellen einer SLURM-Job-Konfiguration genutzt werden.

Welche queues für eine solche Konfiguration am bwUniCluster zur Verfügung stehen und welche Eigenschaften diese Haben kann der Dokumentation unter

https://wiki.bwhpc.de/e/BwUniCluster_2.0_Batch_Queues

entnommen werden.

In [2]:
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(
    queue='dev_multiple', # queue multiple ermöglicht eine Reservierung von mehreren Nodes TODO: remove dev_
    cores=40, # ein Node der queue multiple besitzt 40 cores => für 80 cores werden zwei Nodes angefordert
    memory="90GB", # maximal verfügbarer Speicher pro Node in queue multiple
    local_directory='/tmp',
    walltime='00:30:00',
    interface='ib0' # für die Netzwerkkommunikation im Cluster wollen wir schnelles Infiniband nutzen
)

Der eigentliche Job wird dann auf Basis der Konfiguration mittels der Methode scale gestartet:

In [3]:
cluster.scale(jobs=1) # beim Start der Konfiguration können auch mehrere Jobs gleichzeitig gestartet werden (hierdurch ist das Reservieren mehrerer Nodes möglich)

In [4]:
from dask.distributed import Client
client = Client(cluster)
client

<Client: 'tcp://172.26.21.158:44599' processes=10 threads=80, memory=83.80 GiB>

## Dask Dataframe

Ein Dask Dataframe besteht aus vielen kleinen Pandas DataFrames. Dask DataFrames können auf auf die Festplatten ausgelagert werden um Probleme zu lösen die nicht in den Arbeitsspeicher passen.


<img src="https://docs.dask.org/en/stable/_images/dask-dataframe.svg" alt="Dask DataFrame"
	title="Dask DataFrame" width="300" />

https://docs.dask.org/en/stable/_images/dask-dataframe.svg

In [ ]:
from distributed import Client
client = Client('10.0.1.162:40675')

In [ ]:
client

In [ ]:
import dask.array as da
x = da.random.random((100000,100000), chunks="16 MiB")
x

In [ ]:
y = (x + x.T) - x.mean(axis=0)

In [ ]:
y.sum().compute()

## bwUniCluster Setting

Ein Virtual Environment sollte für eine gute Performance nicht im HOME-Verzeichnis eines Benutzers liegen. Das HOME-Verzeichnis liegt auf einem zentralen File-System (LUSTRE). Für jeden Zugriff ist daher Netzwerkkommunikation notwendig. Dies kann zu Verzögerungen durch Latenzen im Netzwerk führen. Um dies zu umgehen, kann auf jedem Node einzeln ein Environment im temporär verfügbaren File-System des Nodes vorgehalten werden.

```bash
python3 -m pip install s3fs bokeh dask dask_jobqueue ipykernel numpy pandas
```
